In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
import faiss
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
import pandas as pd
import sqlite3
from langchain.prompts import ChatPromptTemplate
from llama_index.llms.groq import Groq

h:\Programs\RAG-LLM_Helpdesk_synthetic-data\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
groq_api_key = os.getenv("GROQ_API_KEY")

In [3]:
SQL_DB_PATH = "../database/csv_database.db"
FAISS_INDEX_PATH = "../database/faiss_index"

In [4]:
# embedding function
def get_embeddings():
    embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
    return embeddings

In [5]:
# LLM model
llm = Groq(model="llama-3.1-8b-instant", api_key=groq_api_key)

In [6]:
# Prompt Template
PROMPT_TEMPLATE ="""
    Jawab pertanyaan berdasarkan konteks dari PDF berikut: 

    {context}

    ---

    **Pertanyaan:**
    {question}

    ---
    Hanya lanjutkan ke bagian di bawah ini jika benar-benar tidak ada informasi spesifik dalam konteks.
    Jika pertanyaan yang diberikan terkait data riwayat penggunaan dan tidak ada informasi spesifik dalam konteks, jawablah dengan **SQL query** yang sesuai dengan contoh tabel berikut:

    **Nama Tabel:** `synthetic_data`

    **Contoh Data dalam Tabel (Format JSON, hanya contoh, tidak digunakan langsung dalam query):**
    ```json
    [
        {{"interaction_id":1,"user_id":447,"timestamp":"2024-03-12 17:56:48","device_type":"Smart Speaker","command_category":"Information","command_text":"Apa rekomendasi restoran di sekitar sini?","ai_response":"Akses ditolak","response_time_ms":310,"ai_confidence_score":67.97,"user_satisfaction":4,"status":"Error","error_code":"ERR403"}},
        {{"interaction_id":2,"user_id":469,"timestamp":"2024-12-01 06:40:13","device_type":"Smartphone","command_category":"Productivity","command_text":"Bagikan agenda meeting dengan anggota tim","ai_response":"Tugas telah disimpan","response_time_ms":315,"ai_confidence_score":70.04,"user_satisfaction":5,"status":"Success","error_code":null}}
    ]
    ```
    Hanya jawab query SQL dalam bentuk string, tanpa memberi jawaban yang lain.

    ---
    Hanya jawab query SQL, jika tidak ada data yang sesuai dalam konteks.
"""

PROMPT_DB = """
    Jawab pertanyaan berdasarkan hasil query SQL yang diberikan di bawah ini:
    {sql_query_result}

    ---
    Jawab pertanyaan berikut:
    {question}

    Jawab tanpa menyebutkan penggunaan query SQL, tetapi parafrase pertanyaan untuk memberi jawaban yang natural.
"""

In [7]:
def query_retrieval(question: str):
    # Load embeddings & vector store
    embeddings = get_embeddings()
    vector_store = FAISS.load_local(FAISS_INDEX_PATH, embeddings, allow_dangerous_deserialization=True)

    # Search vector store
    results = vector_store.similarity_search(question, k=5)
    if not results:
        return "Maaf, saya tidak menemukan informasi yang relevan dalam dokumen."

    context_text = '\n\n---\n\n'.join(doc.page_content for doc in results)

    # Prepare prompt with clear formatting
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=question)

    # Generate response using LLM
    response = llm.complete(prompt)
    response = str(response)

    # Validate if response contains an SQL query
    if "synthetic_data" in response:  
        # Take SQL query from response
        response = response.split("```sql")[1].strip()
        response = response.split("```")[0].strip()

        try:
            # Connect to SQLite database
            conn = sqlite3.connect(SQL_DB_PATH)

            # Execute SQL query safely
            sql_query_result = pd.read_sql_query(response, conn)
            conn.close()

            # If DataFrame is empty, return error message
            if sql_query_result.empty:
                return "Query berhasil dijalankan, tetapi tidak ada hasil yang ditemukan."

            # Convert DataFrame to JSON format string
            query_result_str = sql_query_result.to_json(orient="records")

            # Prepare second-stage prompt (SQL result → natural language)
            prompt_db = ChatPromptTemplate.from_template(PROMPT_DB)
            final_prompt = prompt_db.format(sql_query_result=query_result_str, question=question)

            # Generate final response
            response = llm.complete (final_prompt)

            return response
        
        except sqlite3.Error as e:
            return f"Terjadi kesalahan SQL: {e}"
    
    return response

In [8]:
# prompt
question = 'Berapa rata-rata kepuasan pengguna AI Assistant X-3000 sebelumnya?'
response = query_retrieval(question)
print(response)

Rata-rata kepuasan pengguna AI Assistant X-3000 sebelumnya adalah sekitar 3,44.


In [10]:
# prompt
question = 'Apa saja jenis perangkat yang digunakan oleh pengguna AI Assistant X-3000?'
response = query_retrieval(question)
print(response)

Jenis perangkat yang digunakan oleh pengguna AI Assistant X-3000 adalah Smartphone, Smart Hub, Smart TV, Smart Speaker, dan Laptop.


In [11]:
# prompt
question = 'Apa teknologi yang digunakan untuk pengenalan wajah AI Assistant X-3000?'
response = query_retrieval(question)
print(response)

Teknologi yang digunakan untuk pengenalan wajah AI Assistant X-3000 adalah kamera RGB & IR. Kamera ini bertugas melakukan pengenalan wajah dan pemantauan lingkungan.


In [12]:
# prompt
question = 'Apa fitur utama AI Assistant X-3000?'
response = query_retrieval(question)
print(response)

Fitur utama AI Assistant X-3000 adalah kemampuan memberikan informasi dan rekomendasi yang relevan kepada pengguna, seperti rekomendasi restoran di sekitar lokasi pengguna. Namun, perlu diingat bahwa dalam contoh yang diberikan, AI Assistant X-3000 mengalami kesalahan akses dan tidak dapat memberikan informasi yang diinginkan.


In [13]:
# prompt
question = 'Bagaimana cara troubleshooting jika AI tidak merespons perintah suara?'
response = query_retrieval(question)
print(response)

Jika AI tidak merespons perintah suara, ada beberapa langkah yang dapat dilakukan untuk melakukan troubleshooting:

1. **Periksa koneksi internet**: Pastikan perangkat Anda terhubung ke jaringan internet yang stabil dan cepat. Koneksi internet yang lemah dapat menyebabkan AI tidak dapat merespons perintah suara.
2. **Periksa status perangkat**: Pastikan perangkat Anda dalam keadaan aktif dan siap digunakan. Jika perangkat Anda dalam keadaan mati atau tidak dapat diakses, AI tidak dapat merespons perintah suara.
3. **Periksa pengaturan AI**: Pastikan pengaturan AI dalam keadaan yang tepat. Jika pengaturan AI salah, AI tidak dapat merespons perintah suara.
4. **Periksa perintah suara**: Pastikan perintah suara yang diberikan tepat dan jelas. Jika perintah suara tidak jelas atau salah, AI tidak dapat merespons perintah suara.
5. **Periksa log error**: Periksa log error untuk mengetahui penyebab error yang terjadi. Jika log error menunjukkan error kode "ERR403", maka kemungkinan besar masa

In [14]:
# prompt
question = 'Jelaskan cara menghubungkan AI Assistant X-3000 ke Wi-Fi.'
response = query_retrieval(question)
print(response)

Untuk menghubungkan AI Assistant X-3000 ke Wi-Fi, ikuti langkah-langkah berikut:

1. Pilih opsi "Pengaturan Jaringan" di layar perangkat.
2. AI Assistant X-3000 akan memindai jaringan Wi-Fi yang tersedia.
3. Pilih jaringan yang diinginkan dan masukkan kata sandi.
4. Setelah terhubung, ikon Wi-Fi akan muncul di layar.

Catatan: AI Assistant X-3000 mendukung jaringan Wi-Fi 6 (2.4GHz & 5GHz) untuk koneksi stabil dan cepat. Jika terjadi kegagalan koneksi, pengguna dapat mencoba reset koneksi melalui menu troubleshooting.
